<a href="https://colab.research.google.com/github/EduardoProfe666/Matematica-Numerica-JupyterLab/blob/main/notebooks/Jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from numpy import *
import pandas as pd

# Algoritmo de Jacobi

Algoritmo utilizado para hallar solución a sistemas lineales de orden $n$ en la forma $Ax = b$ con error menor que $\varepsilon$

Hipótesis:
- Se supone que la matriz $A$ posee diagonal predominante

### Clase auxiliar de Resultado


In [2]:
class ResultadoJacobi:
    def __init__(self):
        self.lista_x = []
        self.sigma = 0
        self.error = 0
        self.iteracion_0 = False

### Implementación

``` Jacobi(a, b, x0, f_convergencia, tol, max_iter): ``` Implementación del algoritmo de Jacobi para hallar solución a sistemas de ecuaciones lineales

##### Parámetros

- ``` a ``` : matriz de los coeficientes
- ``` b ``` : matriz de los términos independientes
- ``` x0 ``` : matriz columna que representa los valores estimados de solucio (se puede utilizar la matriz trivial)
- ``` f_convergencia ``` : define el factor de convergencia de la matriz A
- ``` tol ``` : Cota para el error absoluto
- ``` max_iter ```: Cantidad máxima de iteraciones

In [3]:
def jacobi(a, b, x0, f_convergencia, tol, max_iter):
    xv = copy(x0)
    xa = zeros(b.shape[0])
    condition = True
    step = 1
    retorno = []
    r = ResultadoJacobi()

    r.iteracion_0 = True
    r.lista_x = xv.tolist()
    retorno.append(r)

    while condition:
        r = ResultadoJacobi()
        error = 0

        for i in range(a.shape[0]):
            xa[i] = b[i]

            for j in range(a.shape[1]):
                if j != i:
                    xa[i] -= a[i][j] * xv[j]

            xa[i] /= a[i][i]

            if abs(xa[i] - xv[i]) > error:
                error = abs(xa[i] - xv[i])

        r.sigma = error
        error *= abs(f_convergencia / (1 - f_convergencia))
        r.error = error
        r.lista_x = xa.tolist()
        retorno.append(r)

        xv = copy(xa)
        step += 1
        condition = error > tol and step <= max_iter

    return retorno

#### Hallar factor de convergencia

``` hallar_factor_convergencia(a): ``` Halla el factor de convergencia de la matriz a

##### Parámetros
``` a ``` : matriz de los coeficientes

In [4]:
def hallar_factor_convergencia(a):

    a = absolute(a)
    result = []

    for i in range(len(a)):
        total_fila = sum(a[i])

        if total_fila - a[i][i] < 0:
            raise Exception("El factor de convergencia de la matriz es mayor que 0")

        result.append((total_fila - a[i][i]) / a[i][i])

    return max(result)

#### Determinar matriz con diagonal predominante

``` determinar_matriz_diagonal_predominante(a): ``` Determinar si la matriz a tiene diagonal predominante

##### Parámetros
``` a ``` : matriz de los coeficientes

In [5]:
def determinar_matriz_diagonal_predominante(a):
    m = absolute(a)
    for i in range(len(m)):
        x = m[i][i]
        total = sum(m[i]) - x
        if x <= total:
            return False

    return True

### Auxiliares

In [6]:
def hallar_f_convergencia_error(f_convergencia):
    return abs(f_convergencia / (1 - f_convergencia))


def convertir_headers_resultados(lista_resultados_jacobi):
    lista = []

    r = lista_resultados_jacobi[0]
    for i in range(len(r.lista_x)):
        lista.append(f'X{i + 1}')
    lista.append('Sigma')
    lista.append('Error')

    return lista


def convertir_resultados(lista_resultados_jacobi):
    lista = []
    for r in lista_resultados_jacobi:
        l = []
        if r.iteracion_0:
            for x in r.lista_x:
                l.append(x)
            l.append('-------')
            l.append('-------')
        else:
            for x in r.lista_x:
                l.append(x)
            l.append('{:.7f}'.format(r.sigma))
            l.append('{:.7f}'.format(r.error))

        lista.append(l)

    df = pd.DataFrame(data=lista, columns=convertir_headers_resultados(lista_resultados_jacobi))
    df.index.name = 'Iteración'
    return df

### Inserción de datos

In [7]:
a = array([[5, -1, 1],
           [2, 5, -1],
           [-1, 1, 5]])

b = array([10, 12, 10])

x0 = array([0, 0, 0])

tol = 0.005

max_iter = 100

f_convergencia = hallar_factor_convergencia(a)

### Salida de datos

#### Anotaciones Generales

In [8]:
determinante = determinar_matriz_diagonal_predominante(a)
if not determinante:
    print('La matriz proporcionada no tiene diagonal predominante. Haga las transformaciones correspondientes')
else:
    print('La matriz tiene diagonal predominante')
    print(f'El factor de convergencia es {hallar_factor_convergencia(a)}')
    print('ɑ/(1-ɑ) = {:.7f}\n'.format(hallar_f_convergencia_error(hallar_factor_convergencia(a))))
    #
    # print('En la forma X = MX + C, M queda:')
    #
    # print('C queda:')



La matriz tiene diagonal predominante
El factor de convergencia es 0.6
ɑ/(1-ɑ) = 1.5000000



#### Resultados de la aplicación del método de Jacobi

In [10]:
if determinante:
  r = jacobi(a, b, x0, f_convergencia, tol, max_iter)
  df = convertir_resultados(r)

df

,X1,X2,X3,Sigma,Error
Iteración,,,,,
0,0.000000,0.000000,0.000000,-------,-------
1,2.000000,2.400000,2.000000,2.4000000,3.6000000
2,2.080000,2.000000,1.920000,0.4000000,0.6000000
3,2.016000,1.952000,2.016000,0.0960000,0.1440000
4,1.987200,1.996800,2.012800,0.0448000,0.0672000
5,1.996800,2.007680,1.998080,0.0147200,0.0220800
6,2.001920,2.000896,1.997824,0.0067840,0.0101760
7,2.000614,1.998797,2.000205,0.0023808,0.0035712
